# Building Custom Metrics with 3+1 Decomposition

In this example, we'll learn how to build custom spacetime metrics using the 3+1 decomposition formalism. This is a powerful technique for constructing metrics by specifying the lapse function, shift vector, and spatial metric separately.

## What is 3+1 Decomposition?

The 3+1 decomposition splits spacetime into spatial slices and time evolution. Instead of working directly with the 4D metric tensor $g_{\mu\nu}$, we decompose it into three components:

1. **Alpha (α)**: The lapse function - controls the proper time rate
2. **Beta (β)**: The shift vector - describes how spatial coordinates move with time
3. **Gamma (γ)**: The spatial metric - defines the geometry of each spatial slice

The full metric tensor is then:

$$g_{\mu\nu} = \begin{pmatrix}
-\alpha^2 + \beta^i \beta_i & \beta_j \\
\beta_i & \gamma_{ij}
\end{pmatrix}$$

This formalism is particularly useful for numerical relativity and constructing warp drive metrics.

## Import Required Modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from warpfactory.core.tensor import Tensor
from warpfactory.metrics.three_plus_one import (
    set_minkowski_three_plus_one,
    three_plus_one_builder,
    three_plus_one_decomposer
)

## Example 1: Building Flat Minkowski Space

Let's start by building the simplest metric - flat Minkowski space. This will help us understand the structure of the 3+1 components.

In [ ]:
# Define spacetime grid
grid_size = [1, 20, 20, 20]  # [t, x, y, z]

# Get Minkowski space in 3+1 form
alpha, beta, gamma = set_minkowski_three_plus_one(grid_size)

print("3+1 Components of Minkowski Space:")
print(f"\nAlpha (lapse function) shape: {alpha.shape}")
print(f"Alpha value at origin: {alpha[0, 0, 0, 0]}")

print(f"\nBeta (shift vector) has {len(beta)} components (β⁰, β¹, β²)")
for i in range(3):
    print(f"  beta[{i}] shape: {beta[i].shape}, value at origin: {beta[i][0, 0, 0, 0]}")

print(f"\nGamma (spatial metric) has {len(gamma)} components")
for i in range(3):
    for j in range(3):
        print(f"  gamma[({i},{j})] value at origin: {gamma[(i, j)][0, 0, 0, 0]}")

### Understanding the Components

For flat Minkowski space:
- **Alpha = 1** everywhere (proper time flows at coordinate time rate)
- **Beta = 0** everywhere (no coordinate drift)
- **Gamma = δᵢⱼ** (Euclidean spatial metric)

Now let's build the full metric tensor from these components:

In [ ]:
# Build the full 4x4 metric tensor from 3+1 components
metric_dict = three_plus_one_builder(alpha, beta, gamma)

# Create a Tensor object
minkowski_metric = Tensor(
    tensor=metric_dict,
    tensor_type="metric",
    name="Minkowski (from 3+1)",
    index="covariant",
    coords="cartesian",
    scaling=[1.0, 1.0, 1.0, 1.0]
)

print("Built metric tensor components at origin:")
for i in range(4):
    for j in range(4):
        val = minkowski_metric[(i, j)][0, 0, 0, 0]
        print(f"g[{i},{j}] = {val:6.2f}", end="  ")
    print()

print("\nAs expected: diag(-1, 1, 1, 1) for Minkowski space!")

## Example 2: Building a Simple Warp Metric

Now let's create a more interesting metric with a non-trivial shift vector. This will demonstrate how modifying the shift vector β creates a "moving" coordinate system - the basic principle behind warp drive metrics.

In [ ]:
# Start with flat space components
grid_size = [1, 40, 40, 40]
alpha, beta, gamma = set_minkowski_three_plus_one(grid_size)

# Parameters for a simple warp bubble
bubble_center = [0, 20, 20, 20]  # Center of grid
bubble_radius = 8.0
warp_velocity = -0.5  # Negative to move in +x direction

# Create coordinate grids
t_idx, x_idx, y_idx, z_idx = np.meshgrid(
    np.arange(grid_size[0]),
    np.arange(grid_size[1]),
    np.arange(grid_size[2]),
    np.arange(grid_size[3]),
    indexing='ij'
)

# Calculate distance from bubble center
x = x_idx - bubble_center[1]
y = y_idx - bubble_center[2]
z = z_idx - bubble_center[3]
r = np.sqrt(x**2 + y**2 + z**2)

# Create a simple Gaussian shape function
sigma = 2.0
shape_function = np.exp(-(r / sigma)**2)

# Modify the shift vector in the x-direction
# This creates a region of space that "flows" with velocity
beta[0] = warp_velocity * shape_function

print("Created simple warp metric with:")
print(f"  Bubble center: {bubble_center}")
print(f"  Bubble radius: {bubble_radius}")
print(f"  Warp velocity: {warp_velocity}c")
print(f"  Shape function width: {sigma}")
print(f"\nMax shift vector value: {np.max(np.abs(beta[0]))}")

### Visualizing the Shift Vector

Let's visualize the shift vector component we just created. This shows where and how strongly space is being "shifted" by our warp field.

In [ ]:
# Create visualization of the shift vector
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Shift Vector β⁰ (x-component) - Different Slices', fontsize=16)

# XY plane (z = center)
ax = axes[0]
slice_xy = beta[0][0, :, :, grid_size[3]//2]
im = ax.imshow(slice_xy, cmap='RdBu_r', origin='lower', extent=[0, grid_size[1], 0, grid_size[2]])
ax.set_title('XY Plane (z=center)')
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.colorbar(im, ax=ax, label='β⁰')

# XZ plane (y = center)
ax = axes[1]
slice_xz = beta[0][0, :, grid_size[2]//2, :]
im = ax.imshow(slice_xz, cmap='RdBu_r', origin='lower', extent=[0, grid_size[1], 0, grid_size[3]])
ax.set_title('XZ Plane (y=center)')
ax.set_xlabel('x')
ax.set_ylabel('z')
plt.colorbar(im, ax=ax, label='β⁰')

# YZ plane (x = center)
ax = axes[2]
slice_yz = beta[0][0, grid_size[1]//2, :, :]
im = ax.imshow(slice_yz, cmap='RdBu_r', origin='lower', extent=[0, grid_size[2], 0, grid_size[3]])
ax.set_title('YZ Plane (x=center)')
ax.set_xlabel('y')
ax.set_ylabel('z')
plt.colorbar(im, ax=ax, label='β⁰')

plt.tight_layout()
plt.show()

print("The shift vector creates a localized region where spacetime 'flows'.")
print("This is the basic mechanism of warp drive metrics!")

### Building the Complete Warp Metric

Now let's build the full metric tensor from our modified 3+1 components:

In [ ]:
# Build the full metric tensor
warp_metric_dict = three_plus_one_builder(alpha, beta, gamma)

# Create Tensor object
simple_warp_metric = Tensor(
    tensor=warp_metric_dict,
    tensor_type="metric",
    name="Simple Warp Metric",
    index="covariant",
    coords="cartesian",
    scaling=[1.0, 1.0, 1.0, 1.0],
    params={
        "bubble_center": bubble_center,
        "warp_velocity": warp_velocity,
        "shape_sigma": sigma
    }
)

print("Built simple warp metric!")
print(f"Metric name: {simple_warp_metric.name}")
print(f"Parameters: {simple_warp_metric.params}")

### Visualizing the Metric Components

Let's examine how the metric components differ from flat Minkowski space:

In [ ]:
# Compare key metric components at the center slice
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
fig.suptitle('Simple Warp Metric Components (XY plane at center)', fontsize=16)

z_center = grid_size[3] // 2

# g_00 (time-time component)
ax = axes[0, 0]
g00_slice = simple_warp_metric[(0, 0)][0, :, :, z_center]
im = ax.imshow(g00_slice, cmap='RdBu_r', origin='lower')
ax.set_title('$g_{00}$ (time-time)')
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.colorbar(im, ax=ax)

# g_01 (time-x component)
ax = axes[0, 1]
g01_slice = simple_warp_metric[(0, 1)][0, :, :, z_center]
im = ax.imshow(g01_slice, cmap='RdBu_r', origin='lower')
ax.set_title('$g_{01}$ (time-x, = β₀)')
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.colorbar(im, ax=ax)

# g_11 (x-x component)
ax = axes[1, 0]
g11_slice = simple_warp_metric[(1, 1)][0, :, :, z_center]
im = ax.imshow(g11_slice, cmap='RdBu_r', origin='lower')
ax.set_title('$g_{11}$ (x-x)')
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.colorbar(im, ax=ax)

# Difference in g_00 from Minkowski
ax = axes[1, 1]
g00_diff = g00_slice - minkowski_metric[(0, 0)][0, :grid_size[1], :grid_size[2], z_center]
im = ax.imshow(g00_diff, cmap='RdBu_r', origin='lower')
ax.set_title('$g_{00}$ difference from Minkowski')
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.show()

print("Notice how:")
print("  - g_01 = β₀ (the shift vector appears as off-diagonal terms)")
print("  - g_00 = -α² + βⁱβᵢ is modified from -1 where the shift is non-zero")
print("  - g_11 remains 1 (we didn't modify the spatial metric γ)")

## Example 3: Decomposing an Existing Metric

We can also go the other direction - take an existing metric tensor and decompose it into 3+1 components. This is useful for analyzing metrics and understanding their physical properties.

In [ ]:
# Decompose our simple warp metric back into 3+1 form
alpha_decomp, beta_down, gamma_down, beta_up, gamma_up = three_plus_one_decomposer(simple_warp_metric)

print("Decomposed the warp metric back into 3+1 components:")
print(f"\nAlpha at bubble center: {alpha_decomp[0, bubble_center[1], bubble_center[2], bubble_center[3]]:.6f}")
print(f"Original alpha: {alpha[0, bubble_center[1], bubble_center[2], bubble_center[3]]:.6f}")

print(f"\nBeta_0 (covariant) at bubble center: {beta_down[0][0, bubble_center[1], bubble_center[2], bubble_center[3]]:.6f}")
print(f"Original beta[0]: {beta[0][0, bubble_center[1], bubble_center[2], bubble_center[3]]:.6f}")

print(f"\nGamma_00 at bubble center: {gamma_down[(0,0)][0, bubble_center[1], bubble_center[2], bubble_center[3]]:.6f}")
print(f"Original gamma[(0,0)]: {gamma[(0,0)][0, bubble_center[1], bubble_center[2], bubble_center[3]]:.6f}")

print("\nThe decomposer successfully recovered the original 3+1 components!")

## Example 4: Building with a Non-Trivial Spatial Metric

So far we've only modified the shift vector. Now let's create a metric with a modified spatial metric γ as well. This allows us to create actual spatial curvature, not just coordinate flow.

In [ ]:
# Start with flat space
grid_size = [1, 30, 30, 30]
alpha, beta, gamma = set_minkowski_three_plus_one(grid_size)

# Create a spatial expansion/contraction
expansion_center = [0, 15, 15, 15]
expansion_sigma = 3.0
expansion_strength = 0.2

# Create coordinate grids
t_idx, x_idx, y_idx, z_idx = np.meshgrid(
    np.arange(grid_size[0]),
    np.arange(grid_size[1]),
    np.arange(grid_size[2]),
    np.arange(grid_size[3]),
    indexing='ij'
)

x = x_idx - expansion_center[1]
y = y_idx - expansion_center[2]
z = z_idx - expansion_center[3]
r = np.sqrt(x**2 + y**2 + z**2)

# Gaussian expansion factor
expansion_factor = 1.0 + expansion_strength * np.exp(-(r / expansion_sigma)**2)

# Modify the spatial metric (make space "stretch" near the center)
gamma[(0, 0)] = expansion_factor  # x-x component
gamma[(1, 1)] = expansion_factor  # y-y component  
gamma[(2, 2)] = expansion_factor  # z-z component

print("Created metric with modified spatial geometry:")
print(f"  Expansion center: {expansion_center}")
print(f"  Expansion strength: {expansion_strength}")
print(f"  Max gamma_xx value: {np.max(gamma[(0,0)]):.4f}")
print(f"  Min gamma_xx value: {np.min(gamma[(0,0)]):.4f}")

In [ ]:
# Build the metric with modified spatial geometry
curved_metric_dict = three_plus_one_builder(alpha, beta, gamma)

curved_metric = Tensor(
    tensor=curved_metric_dict,
    tensor_type="metric",
    name="Spatially Curved Metric",
    index="covariant",
    coords="cartesian",
    scaling=[1.0, 1.0, 1.0, 1.0],
    params={
        "expansion_center": expansion_center,
        "expansion_strength": expansion_strength,
        "expansion_sigma": expansion_sigma
    }
)

# Visualize the spatial metric component
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('Modified Spatial Metric γ₀₀ (x-x component)', fontsize=16)

# XY slice
ax = axes[0]
gamma_slice = gamma[(0, 0)][0, :, :, grid_size[3]//2]
im = ax.imshow(gamma_slice, cmap='viridis', origin='lower')
ax.set_title('XY Plane (z=center)')
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.colorbar(im, ax=ax, label='γ₀₀')

# Radial profile
ax = axes[1]
center_line = gamma[(0, 0)][0, :, grid_size[2]//2, grid_size[3]//2]
ax.plot(center_line, linewidth=2)
ax.set_title('Radial Profile along x-axis')
ax.set_xlabel('x index')
ax.set_ylabel('γ₀₀')
ax.grid(True, alpha=0.3)
ax.axhline(y=1.0, color='r', linestyle='--', label='Flat space (γ=1)')
ax.legend()

plt.tight_layout()
plt.show()

print("\nThe spatial metric is expanded (γ > 1) near the center.")
print("This represents genuine spatial curvature, not just coordinate effects!")

## Example 5: Combining All Components

Finally, let's create a complex metric that modifies all three components: alpha, beta, and gamma. This demonstrates the full flexibility of the 3+1 formalism.

In [ ]:
# Start fresh
grid_size = [1, 40, 40, 40]
alpha, beta, gamma = set_minkowski_three_plus_one(grid_size)

# Parameters
center = [0, 20, 20, 20]
sigma = 3.0

# Create coordinate grids
t_idx, x_idx, y_idx, z_idx = np.meshgrid(
    np.arange(grid_size[0]),
    np.arange(grid_size[1]),
    np.arange(grid_size[2]),
    np.arange(grid_size[3]),
    indexing='ij'
)

x = x_idx - center[1]
y = y_idx - center[2]
z = z_idx - center[3]
r = np.sqrt(x**2 + y**2 + z**2)

# Shape function
f = np.exp(-(r / sigma)**2)

# 1. Modify lapse function (time dilation effect)
time_dilation = 0.15
alpha = 1.0 - time_dilation * f

# 2. Modify shift vector (coordinate flow)
velocity = -0.3
beta[0] = velocity * f  # x-direction flow

# 3. Modify spatial metric (spatial expansion)
expansion = 0.2
gamma[(0, 0)] = 1.0 + expansion * f  # x-x
gamma[(1, 1)] = 1.0 + expansion * f  # y-y
gamma[(2, 2)] = 1.0 + expansion * f  # z-z

print("Created complex metric with all components modified:")
print(f"  Time dilation: {time_dilation} (α < 1 means time slows down)")
print(f"  Coordinate velocity: {velocity}c")
print(f"  Spatial expansion: {expansion} (γ > 1 means space is stretched)")
print(f"\nAt the center:")
print(f"  Alpha: {alpha[0, center[1], center[2], center[3]]:.4f}")
print(f"  Beta[0]: {beta[0][0, center[1], center[2], center[3]]:.4f}")
print(f"  Gamma[0,0]: {gamma[(0,0)][0, center[1], center[2], center[3]]:.4f}")

In [ ]:
# Build the complete complex metric
complex_metric_dict = three_plus_one_builder(alpha, beta, gamma)

complex_metric = Tensor(
    tensor=complex_metric_dict,
    tensor_type="metric",
    name="Complex 3+1 Metric",
    index="covariant",
    coords="cartesian",
    scaling=[1.0, 1.0, 1.0, 1.0],
    params={
        "center": center,
        "sigma": sigma,
        "time_dilation": time_dilation,
        "velocity": velocity,
        "spatial_expansion": expansion
    }
)

# Visualize all three components
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('All 3+1 Components (XY plane at center)', fontsize=16)

z_center = grid_size[3] // 2

# Alpha
ax = axes[0]
alpha_slice = alpha[0, :, :, z_center]
im = ax.imshow(alpha_slice, cmap='plasma', origin='lower')
ax.set_title('α (Lapse Function)')
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.colorbar(im, ax=ax, label='α')

# Beta
ax = axes[1]
beta_slice = beta[0][0, :, :, z_center]
im = ax.imshow(beta_slice, cmap='RdBu_r', origin='lower')
ax.set_title('β⁰ (Shift Vector x-component)')
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.colorbar(im, ax=ax, label='β⁰')

# Gamma
ax = axes[2]
gamma_slice = gamma[(0, 0)][0, :, :, z_center]
im = ax.imshow(gamma_slice, cmap='viridis', origin='lower')
ax.set_title('γ₀₀ (Spatial Metric x-x)')
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.colorbar(im, ax=ax, label='γ₀₀')

plt.tight_layout()
plt.show()

print("\nThis metric combines:")
print("  - Time dilation (varying α)")
print("  - Coordinate flow (non-zero β)")
print("  - Spatial curvature (modified γ)")
print("\nThis is the type of metric used in advanced warp drive designs!")

## Summary and Key Takeaways

In this notebook, we learned how to build custom spacetime metrics using the 3+1 decomposition:

### Key Functions:
1. **`set_minkowski_three_plus_one(grid_size)`** - Creates flat space 3+1 components as a starting point
2. **`three_plus_one_builder(alpha, beta, gamma)`** - Builds the full 4x4 metric tensor from components
3. **`three_plus_one_decomposer(metric)`** - Decomposes an existing metric into 3+1 components

### The 3+1 Components:
- **Alpha (α)**: Lapse function - scalar field controlling proper time flow
  - α < 1: Time flows slower (time dilation)
  - α = 1: Normal time flow (Minkowski)
  - α > 1: Time flows faster

- **Beta (βⁱ)**: Shift vector - 3-component vector field
  - Creates coordinate "flow" or "drift"
  - Essential for warp drive metrics
  - Appears as off-diagonal terms g₀ᵢ in the metric

- **Gamma (γᵢⱼ)**: Spatial metric - 3×3 symmetric tensor
  - Defines the geometry of spatial slices
  - γᵢⱼ = δᵢⱼ for flat space
  - Modified γ creates genuine spatial curvature

### Practical Applications:
- Building warp drive metrics (Alcubierre, Lentz, etc.)
- Numerical relativity simulations
- Analyzing spacetime structure
- Creating custom exotic spacetimes

### Next Steps:
Try creating your own custom metrics! Experiment with different shape functions for α, β, and γ to see how they affect the spacetime geometry. You can then use the energy tensor calculations from other examples to check if your metric satisfies the energy conditions.